# Linear Gaussian test  
deismic

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sbi.simulators as simulators
import sbi.utils as utils
import torch
from sbi.inference.snpe.snpe_c import SnpeC
from torch import distributions

# use cpu by default
torch.set_default_tensor_type("torch.FloatTensor")

# seed the simulations
torch.manual_seed(0)

## Create NN instance

In [ ]:
dim, std = 3, 0.5
simulator = simulators.LinearGaussianSimulator(dim=dim, std=std)
prior = distributions.MultivariateNormal(loc=torch.zeros(dim), covariance_matrix=torch.eye(dim))

In [ ]:
ground_truth_parameters = prior.sample()
true_observation = simulator(ground_truth_parameters) 

In [ ]:
neural_posterior = utils.get_sbi_posterior(
    model='maf',
    prior=prior,
    context=true_observation,
)

In [ ]:
apt = SnpeC(
    simulator=simulator,
    true_observation=true_observation,
    prior=prior,
    density_estimator=neural_posterior,
    num_atoms=-1,
    z_score_obs=True,
    use_combined_loss=False,
    retrain_from_scratch_each_round=False,
    discard_prior_samples=False,
)

### Run inference

In [ ]:
num_rounds, num_simulations_per_round = 1, 2000
apt.run_inference(
    num_rounds=num_rounds, num_simulations_per_round=num_simulations_per_round
)

### Draw samples and plot hist

In [ ]:
samples = apt._neural_posterior.sample(2500)
samples = utils.tensor2numpy(samples)
figure = utils.plot_hist_marginals(
    data=samples,
    ground_truth=utils.tensor2numpy(
        ground_truth_parameters
    ).reshape(-1),
    lims=[-4, 4],
)